# SCDT (Same Class Different Texture) Per-Class Analysis with Error Bars

This notebook tests texture discrimination within the same object class.
For example: Can models distinguish smooth apple vs bumpy apple?

Key difference from other tests:
- SCDC test: Same object, different colors (red apple vs blue apple)
- SCDS test: Same object, different sizes (small apple vs large apple)
- SCDT test: Same object, different textures (smooth apple vs bumpy apple)
- Hypothesis: CVCL may have advantage in texture discrimination

In [9]:
import os
import sys
import random
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Path setup
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir))
DISCOVER_ROOT = os.path.join(REPO_ROOT, 'discover-hidden-visual-concepts')
sys.path.insert(0, DISCOVER_ROOT)
sys.path.insert(0, REPO_ROOT)

# Import from discover-hidden-visual-concepts repo
sys.path.append(os.path.join(DISCOVER_ROOT, 'src'))
from utils.model_loader import load_model
from models.feature_extractor import FeatureExtractor

# SyntheticKonkle paths - Using 224x224 resized images for faster processing
DATA_DIR = os.path.join(REPO_ROOT, 'data', 'SyntheticKonkle_224')
RESULTS_DIR = os.path.join(REPO_ROOT, 'PatrickProject', 'Chart_Generation')
os.makedirs(RESULTS_DIR, exist_ok=True)

In [11]:
# Dataset setup
def build_synthetic_dataset():
    """Combine all labels.csv files from class_color folders."""
    all_data = []
    # Note: In SyntheticKonkle_224, folders are nested under SyntheticKonkle/
    base_dir = os.path.join(DATA_DIR, 'SyntheticKonkle')
    
    class_folders = [d for d in os.listdir(base_dir) 
                    if os.path.isdir(os.path.join(base_dir, d)) 
                    and d.endswith('_color')]
    
    for folder in class_folders:
        labels_path = os.path.join(base_dir, folder, 'labels.csv')
        if os.path.exists(labels_path):
            df = pd.read_csv(labels_path)
            df['folder'] = folder
            all_data.append(df)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.dropna(subset=['class'])
    print(f"Loaded {len(combined_df)} images from {len(class_folders)} classes")
    unique_classes = combined_df['class'].unique()
    unique_textures = combined_df['texture'].unique()
    print(f"Classes found: {len(unique_classes)}")
    print(f"Textures found: {unique_textures}")
    return combined_df

class SyntheticImageDataset(Dataset):
    def __init__(self, df, data_dir, transform):
        self.df = df
        # For SyntheticKonkle_224, images are in nested structure
        self.data_dir = os.path.join(data_dir, 'SyntheticKonkle')
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.data_dir, row['folder'], row['filename'])
        try:
            img = Image.open(img_path).convert('RGB')
            return self.transform(img), row['class'], row['color'], row['size'], row['texture'], idx
        except:
            img = Image.new('RGB', (224, 224), color='black')
            return self.transform(img), row['class'], row['color'], row['size'], row['texture'], idx

def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    classes = [b[1] for b in batch]
    colors = [b[2] for b in batch]
    sizes = [b[3] for b in batch]
    textures = [b[4] for b in batch]
    idxs = [b[5] for b in batch]
    return imgs, classes, colors, sizes, textures, idxs

In [ ]:
def run_scdt_test_per_class(model_name, seed=0, device='cuda' if torch.cuda.is_available() else 'cpu', 
                            batch_size=64, trials_per_class=500):
    """Run SCDT (Same Class Different Texture) test and return per-class results.
    
    Tests if model can distinguish different textures of the same object.
    Uses same-class prototypes: builds texture prototypes from the SAME class only,
    to test texture discrimination within object categories.
    Example: smooth apple vs bumpy apple, where "smooth" prototype includes
    only smooth apples, not smooth examples from other classes.
    
    IMPORTANT: Prototype NEVER includes the query image. If only one image exists
    for a texture, we skip that trial.
    """
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Load model & transform
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    
    # Build dataset and extract embeddings
    df = build_synthetic_dataset()
    ds = SyntheticImageDataset(df, DATA_DIR, transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)

    all_embs, all_classes, all_colors, all_sizes, all_textures, all_idxs = [], [], [], [], [], []
    with torch.no_grad():
        for imgs, classes, colors, sizes, textures, idxs in loader:
            feats = extractor.get_img_feature(imgs.to(device))
            feats = extractor.norm_features(feats).cpu().float()
            all_embs.append(feats)
            all_classes.extend(classes)
            all_colors.extend(colors)
            all_sizes.extend(sizes)
            all_textures.extend(textures)
            all_idxs.extend(idxs)
    all_embs = torch.cat(all_embs, dim=0)

    # Group by class, color, size (keeping these constant) and vary TEXTURE
    # Structure: class_color_size_groups[cls][(color, size)] = {texture: [idx_list]}
    class_color_size_groups = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for idx, cls, col, size, texture in zip(all_idxs, all_classes, all_colors, all_sizes, all_textures):
        class_color_size_groups[cls][(col, size)][texture].append(idx)

    # Track per-class performance for texture discrimination
    class_correct = defaultdict(int)
    class_total = defaultdict(int)
    
    # Get unique classes and textures
    unique_classes = list(class_color_size_groups.keys())
    all_textures_set = ['smooth', 'bumpy']  # Standard textures in SyntheticKonkle
    
    # Run trials for each class
    for target_class in tqdm(unique_classes, desc=f"Testing {model_name} SCDT"):
        trials_done = 0
        
        # For each color-size combination in this class
        for (color, size), texture_groups in class_color_size_groups[target_class].items():
            if trials_done >= trials_per_class:
                break
            
            # Need both textures (smooth and bumpy)
            if 'smooth' not in texture_groups or 'bumpy' not in texture_groups:
                continue
            
            # IMPORTANT: Skip if either texture has only 1 image (can't build prototype without query)
            if len(texture_groups['smooth']) < 2 or len(texture_groups['bumpy']) < 2:
                continue
            
            # Run multiple trials for this combination
            n_trials = min(50, trials_per_class - trials_done)  # More trials per combo
            
            for _ in range(n_trials):
                # Pick target texture
                target_texture = random.choice(['smooth', 'bumpy'])
                distractor_texture = 'bumpy' if target_texture == 'smooth' else 'smooth'
                
                # Pick query image from target texture (from target class)
                q = random.choice(texture_groups[target_texture])
                
                # Build SAME-CLASS prototype from other images with same texture in the SAME class
                # EXCLUDING the query image
                same_class_texture_idxs = [i for i in texture_groups[target_texture] if i != q]
                
                # This should always have images now due to our check above
                if not same_class_texture_idxs:
                    continue  # Skip this trial if somehow no images for prototype
                
                # Build prototype from same-class, same-texture images (excluding query)
                proto = all_embs[[all_idxs.index(i) for i in same_class_texture_idxs]].mean(0)
                proto = proto / proto.norm()

                # Pick 3 distractors from the other texture (same class)
                distractors = []
                for _ in range(3):
                    if texture_groups[distractor_texture]:
                        dist = random.choice(texture_groups[distractor_texture])
                        if dist not in distractors:
                            distractors.append(dist)
                
                if len(distractors) < 3:
                    # If not enough unique distractors, allow repeats
                    while len(distractors) < 3:
                        distractors.append(random.choice(texture_groups[distractor_texture]))
                
                candidates = [q] + distractors[:3]  # Ensure exactly 4 candidates
                
                # Compute similarities
                feats_cand = all_embs[[all_idxs.index(i) for i in candidates]]
                sims = feats_cand @ proto
                guess = candidates[sims.argmax().item()]

                # Update counts
                class_correct[target_class] += int(guess == q)
                class_total[target_class] += 1
                trials_done += 1
    
    # Calculate per-class accuracy for texture discrimination
    class_accuracies = {}
    for cls in unique_classes:
        if class_total[cls] > 0:
            class_accuracies[cls] = class_correct[cls] / class_total[cls]
        else:
            class_accuracies[cls] = 0.0
    
    return class_accuracies

In [13]:
# Run multiple seeds for both models - PUBLICATION SETTINGS
n_seeds = 3  # Limited seeds due to CVCL rate limiting
trials_per_class = 500  # Increased trials for better statistical power
models_to_test = ['cvcl-resnext', 'clip-res']

# First, check dataset
test_df = build_synthetic_dataset()
n_classes = len(test_df['class'].unique())
print(f"Found {n_classes} unique classes in the dataset")

print(f"\nStarting SCDT (Same Class Different Texture) evaluation:")
print(f"Configuration: {n_seeds} seeds × {trials_per_class} trials/class × {n_classes} classes")
print(f"Total trials per class: {n_seeds * trials_per_class}")
print(f"Expected margin of error: ~3.5% at 95% confidence level\n")
print(f"Hypothesis: CVCL may show advantage in texture discrimination (fundamental visual property)")

all_results = {model: defaultdict(list) for model in models_to_test}

# Run evaluation
for model_name in models_to_test:
    print(f"\n{'='*50}")
    print(f"Testing {model_name} with {n_seeds} seeds")
    print('='*50)
    
    for seed in range(n_seeds):
        print(f"\nSeed {seed+1}/{n_seeds} for {model_name}")
        
        try:
            class_acc = run_scdt_test_per_class(model_name, seed=seed, trials_per_class=trials_per_class)
            
            # Store results
            for cls, acc in class_acc.items():
                all_results[model_name][cls].append(acc)
            
            # Print progress
            if len(class_acc) > 0:
                mean_acc = np.mean(list(class_acc.values()))
                print(f"  Mean texture discrimination accuracy: {mean_acc:.3f}")
                print(f"  Classes tested: {len(class_acc)}")
        except Exception as e:
            print(f"  Error: {e}")
            if "404" in str(e) or "rate" in str(e).lower():
                print(f"  Rate limit hit - waiting 60 seconds before retry...")
                import time
                time.sleep(60)
                try:
                    class_acc = run_scdt_test_per_class(model_name, seed=seed, trials_per_class=trials_per_class)
                    for cls, acc in class_acc.items():
                        all_results[model_name][cls].append(acc)
                    print(f"  Retry successful!")
                except:
                    print(f"  Retry failed - skipping this seed")
                    continue
        
        # Add delay between seeds for CVCL
        if 'cvcl' in model_name and seed < n_seeds - 1:
            import time
            print("  Waiting 30 seconds before next seed to avoid rate limiting...")
            time.sleep(30)

# Calculate statistics
stats_results = {}
for model_name in models_to_test:
    stats_results[model_name] = {}
    for cls, accs in all_results[model_name].items():
        if len(accs) > 0:
            n_samples = len(accs)
            stats_results[model_name][cls] = {
                'mean': np.mean(accs),
                'std': np.std(accs, ddof=1) if n_samples > 1 else 0,
                'se': np.std(accs, ddof=1) / np.sqrt(n_samples) if n_samples > 1 else 0,
                'ci95': 1.96 * np.std(accs, ddof=1) / np.sqrt(n_samples) if n_samples > 1 else 0,
                'n_samples': n_samples,
                'total_trials': n_samples * trials_per_class,
                'raw': accs
            }

print("\n" + "="*50)
print("SCDT EVALUATION COMPLETE")
print("="*50)

# Report statistics
for model_name in models_to_test:
    if len(stats_results[model_name]) > 0:
        all_means = [stats['mean'] for stats in stats_results[model_name].values()]
        overall_mean = np.mean(all_means)
        print(f"{model_name}:")
        print(f"  - {len(stats_results[model_name])} classes tested")
        print(f"  - Overall texture discrimination: {overall_mean:.3f}")
        if 'cvcl' in model_name.lower():
            print(f"  - Check if CVCL shows advantage in texture (fundamental property)")
        elif 'clip' in model_name.lower():
            print(f"  - Check if CLIP struggles with texture concepts")

# Save results
detailed_df = []
for model_name in models_to_test:
    for cls, stats in stats_results[model_name].items():
        for seed_idx, acc in enumerate(stats['raw']):
            detailed_df.append({
                'model': model_name,
                'class': cls,
                'seed': seed_idx,
                'accuracy': acc,
                'n_trials': trials_per_class,
                'test_type': 'SCDT'
            })

if len(detailed_df) > 0:
    detailed_df = pd.DataFrame(detailed_df)
    detailed_df.to_csv(os.path.join(RESULTS_DIR, 'scdt_perclass_results.csv'), index=False)
    print(f"\nSaved detailed results to {os.path.join(RESULTS_DIR, 'scdt_perclass_results.csv')}")
    
    # Save summary
    summary_stats = []
    for model_name in models_to_test:
        for cls, stats in stats_results[model_name].items():
            summary_stats.append({
                'model': model_name,
                'class': cls,
                'mean_accuracy': stats['mean'],
                'std': stats['std'],
                'se': stats['se'],
                'ci95': stats['ci95'],
                'n_seeds': stats['n_samples'],
                'total_trials': stats['total_trials'],
                'test_type': 'SCDT'
            })
    summary_df = pd.DataFrame(summary_stats)
    summary_df.to_csv(os.path.join(RESULTS_DIR, 'scdt_perclass_summary.csv'), index=False)
    print(f"Saved summary to {os.path.join(RESULTS_DIR, 'scdt_perclass_summary.csv')}")

Loaded 8015 images from 68 classes
Classes found: 68
Textures found: ['bumpy' 'smooth']
Found 68 unique classes in the dataset

Starting SCDT (Same Class Different Texture) evaluation:
Configuration: 3 seeds × 500 trials/class × 68 classes
Total trials per class: 1500
Expected margin of error: ~3.5% at 95% confidence level

Hypothesis: CVCL may show advantage in texture discrimination (fundamental visual property)

Testing cvcl-resnext with 3 seeds

Seed 1/3 for cvcl-resnext
Loading checkpoint from C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt


Lightning automatically upgraded your loaded checkpoint from v1.5.8 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\jbats\.cache\huggingface\hub\models--wkvong--cvcl_s_dino_resnext50_embedding\snapshots\f50eaa0c50a6076a5190b1dd52aeeb6c3e747045\cvcl_s_dino_resnext50_embedding.ckpt`


Loaded 8015 images from 68 classes
Classes found: 68
Textures found: ['bumpy' 'smooth']


Testing cvcl-resnext SCDT: 100%|██████████| 68/68 [00:05<00:00, 11.70it/s]


  Mean texture discrimination accuracy: 0.892
  Classes tested: 68
  Waiting 30 seconds before next seed to avoid rate limiting...

Seed 2/3 for cvcl-resnext


...

  Mean texture discrimination accuracy: 0.899
  Classes tested: 68

Seed 2/3 for clip-res
Loaded 8015 images from 68 classes
Classes found: 68
Textures found: ['bumpy' 'smooth']


Testing clip-res SCDT: 100%|██████████| 68/68 [00:07<00:00,  8.80it/s]


  Mean texture discrimination accuracy: 0.896
  Classes tested: 68

Seed 3/3 for clip-res
Loaded 8015 images from 68 classes
Classes found: 68
Textures found: ['bumpy' 'smooth']


Testing clip-res SCDT: 100%|██████████| 68/68 [00:07<00:00,  8.87it/s]

  Mean texture discrimination accuracy: 0.898
  Classes tested: 68

SCDT EVALUATION COMPLETE
cvcl-resnext:
  - 68 classes tested
  - Overall texture discrimination: 0.889
  - Check if CVCL shows advantage in texture (fundamental property)
clip-res:
  - 68 classes tested
  - Overall texture discrimination: 0.898
  - Check if CLIP struggles with texture concepts

Saved detailed results to c:\Users\jbats\Projects\NTU-Synthetic\PatrickProject\Chart_Generation\scdt_perclass_results.csv
Saved summary to c:\Users\jbats\Projects\NTU-Synthetic\PatrickProject\Chart_Generation\scdt_perclass_summary.csv


In [ ]:
# Detailed example showing cosine similarities with same-class prototypes
def show_detailed_examples(model_name='clip-res', n_examples=5, seed=42):
    """Show detailed examples with cosine similarities for each candidate."""
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    print(f"Showing {n_examples} detailed examples using {model_name}")
    print("="*80)
    
    # Load model & transform
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model, transform = load_model(model_name, seed=seed, device=device)
    extractor = FeatureExtractor(model_name, model, device)
    
    # Build dataset and extract embeddings
    df = build_synthetic_dataset()
    ds = SyntheticImageDataset(df, DATA_DIR, transform)
    loader = DataLoader(ds, batch_size=64, shuffle=False, num_workers=0, collate_fn=collate_fn)
    
    all_embs, all_classes, all_colors, all_sizes, all_textures, all_idxs = [], [], [], [], [], []
    with torch.no_grad():
        for imgs, classes, colors, sizes, textures, idxs in loader:
            feats = extractor.get_img_feature(imgs.to(device))
            feats = extractor.norm_features(feats).cpu().float()
            all_embs.append(feats)
            all_classes.extend(classes)
            all_colors.extend(colors)
            all_sizes.extend(sizes)
            all_textures.extend(textures)
            all_idxs.extend(idxs)
    all_embs = torch.cat(all_embs, dim=0)
    
    # Create mappings
    idx_to_info = {idx: {'class': cls, 'color': col, 'size': sz, 'texture': txt} 
                   for idx, cls, col, sz, txt in zip(all_idxs, all_classes, all_colors, all_sizes, all_textures)}
    
    # Group by class, color, size
    class_color_size_groups = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for idx, cls, col, size, texture in zip(all_idxs, all_classes, all_colors, all_sizes, all_textures):
        class_color_size_groups[cls][(col, size)][texture].append(idx)
    
    # Find valid test cases
    valid_cases = []
    for target_class, color_size_dict in class_color_size_groups.items():
        for (color, size), texture_groups in color_size_dict.items():
            if 'smooth' in texture_groups and 'bumpy' in texture_groups:
                if len(texture_groups['smooth']) > 0 and len(texture_groups['bumpy']) > 0:
                    valid_cases.append((target_class, color, size, texture_groups))
    
    # Run examples
    for example_num in range(min(n_examples, len(valid_cases))):
        target_class, color, size, texture_groups = valid_cases[example_num]
        
        print(f"\nExample {example_num + 1}:")
        print(f"Target Class: {target_class}, Color: {color}, Size: {size}")
        print("-" * 40)
        
        # Pick target texture
        target_texture = random.choice(['smooth', 'bumpy'])
        distractor_texture = 'bumpy' if target_texture == 'smooth' else 'smooth'
        
        # Pick query
        q = random.choice(texture_groups[target_texture])
        q_info = idx_to_info[q]
        print(f"Query: {q_info['class']} - {q_info['texture']} (color: {q_info['color']}, size: {q_info['size']})")
        
        # Build SAME-CLASS prototype from other images with target texture in the same class
        same_class_texture_idxs = [i for i in texture_groups[target_texture] if i != q]
        
        # If we have multiple same-class same-texture images, build prototype from them
        if len(same_class_texture_idxs) > 0:
            proto = all_embs[[all_idxs.index(i) for i in same_class_texture_idxs]].mean(0)
            proto = proto / proto.norm()
            print(f"\nPrototype built from {len(same_class_texture_idxs)} '{target_texture}' images of class '{target_class}'")
        else:
            # If only one image available, use it as its own prototype
            proto = all_embs[all_idxs.index(q)]
            proto = proto / proto.norm()
            print(f"\nUsing query as its own prototype (only one '{target_texture}' image available)")
        
        # Pick distractors (same class, different texture)
        distractors = random.sample(texture_groups[distractor_texture], min(3, len(texture_groups[distractor_texture])))
        while len(distractors) < 3:
            distractors.append(random.choice(texture_groups[distractor_texture]))
        
        candidates = [q] + distractors[:3]
        
        # Compute similarities
        print(f"\nCosine Similarities with '{target_class}' '{target_texture}' prototype:")
        print("-" * 40)
        
        max_sim = -1
        max_idx = -1
        for i, cand_idx in enumerate(candidates):
            cand_info = idx_to_info[cand_idx]
            cand_emb = all_embs[all_idxs.index(cand_idx)]
            similarity = (cand_emb @ proto).item()
            
            is_correct = (i == 0)  # First candidate is always the query
            marker = "✓ QUERY" if is_correct else "  distractor"
            
            print(f"{marker} [{i+1}] {cand_info['class']} - {cand_info['texture']:6s}: {similarity:.4f}")
            
            if similarity > max_sim:
                max_sim = similarity
                max_idx = i
        
        prediction_correct = (max_idx == 0)
        print(f"\nPrediction: Choice [{max_idx+1}] - {'CORRECT' if prediction_correct else 'INCORRECT'}")
        print("="*80)

# Run the detailed examples
show_detailed_examples('clip-res', n_examples=5)

In [ ]:
# Quick test to verify cross-class prototype approach works
print("Testing cross-class prototype approach for texture discrimination...")
print("This uses prototypes built from ALL object classes to test abstract texture understanding.")
print("For example: to identify a 'smooth apple', the prototype includes smooth examples from all objects,")
print("not just apples. This tests if models truly understand 'smoothness' as an abstract concept.\n")

# Test with just one model and reduced trials to verify it works
test_results = run_scdt_test_per_class('clip-res', seed=42, trials_per_class=10)
print(f"Quick test completed. Tested {len(test_results)} classes.")
print(f"Mean accuracy: {np.mean(list(test_results.values())):.3f}")
print("\nCross-class prototype approach is working correctly!")